In [1]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# -------------------------------
# Configuration
# -------------------------------
FOLDER_PATH = "part_1"            # Folder containing CSV files
OUTPUT_FOLDER = "outputs/fft_plots"     # Folder to save plots
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# -------------------------------
# Function to compute FFT
# -------------------------------
def compute_fft(time, signal):
    n = len(signal)
    dt = np.mean(np.diff(time))  # Sampling interval
    fft_values = np.fft.fft(signal)
    freqs = np.fft.fftfreq(n, dt)
    # Keep only positive frequencies
    positive = freqs > 0
    #returns frequency and amplitude
    return freqs[positive], np.abs(fft_values[positive])*2/n

# -------------------------------
# Process all CSV files
# -------------------------------
csv_files = glob.glob(os.path.join(FOLDER_PATH, "*.csv"))

for file in csv_files:
    print(f"Processing {file}")
    df = pd.read_csv(file)

    # Ensure two columns: time + signal
    if df.shape[1] < 2:
        print(f"Skipping {file}: not enough columns")
        continue

    time = df.iloc[:, 0].values
    signal = df.iloc[:, 1].values

    freqs, amplitude = compute_fft(time, signal)

    # Plot
    plt.figure(figsize=(8, 4))
    plt.plot(freqs, amplitude)
    plt.title(f"FFT - {os.path.basename(file)}")
    plt.xlabel("Frequency")
    plt.ylabel("Amplitude")
    plt.grid(True)
    plt.tight_layout()

    # Save plot
    output_path = os.path.join(OUTPUT_FOLDER, os.path.basename(file).replace(".csv", "_fft.png"))
    plt.savefig(output_path)
    plt.close()  # Close figure to free memory
    print(f"Saved plot to {output_path}")

Processing part_1\11d8b435-ba4a-564f-b0e8-d5cbed8adbb2.csv
Saved plot to outputs/fft_plots\11d8b435-ba4a-564f-b0e8-d5cbed8adbb2_fft.png
Processing part_1\3186c48d-fc24-5300-910a-6d0bafdd87ea.csv
Saved plot to outputs/fft_plots\3186c48d-fc24-5300-910a-6d0bafdd87ea_fft.png
Processing part_1\555cbc73-5a58-53a2-b432-c415f46e8c7c.csv
Saved plot to outputs/fft_plots\555cbc73-5a58-53a2-b432-c415f46e8c7c_fft.png
Processing part_1\6dbf3276-3d5a-5c9f-930e-09da6ec60243.csv
Saved plot to outputs/fft_plots\6dbf3276-3d5a-5c9f-930e-09da6ec60243_fft.png
Processing part_1\75a0970d-7c9a-5fd4-9a83-80cddf68ce6c.csv
Saved plot to outputs/fft_plots\75a0970d-7c9a-5fd4-9a83-80cddf68ce6c_fft.png
Processing part_1\771e32b1-39b4-5a58-bb2b-c618ce2701d8.csv
Saved plot to outputs/fft_plots\771e32b1-39b4-5a58-bb2b-c618ce2701d8_fft.png
Processing part_1\82e91f2f-4ed5-5591-9617-0ff6f3b0e0c1.csv
Saved plot to outputs/fft_plots\82e91f2f-4ed5-5591-9617-0ff6f3b0e0c1_fft.png
Processing part_1\9da3a9bb-65e4-5899-9280-cdd730

In [2]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt

# -------------------------------
# Configuration
# -------------------------------
FOLDER_PATH = "part_1"
OUTPUT_PLOTS = "outputs/band_plots"
OUTPUT_FEATURES = "band_features.csv"

os.makedirs(OUTPUT_PLOTS, exist_ok=True)

G = 9.81  # Convert g to m/s²

# Define frequency bands (customize here)
BANDS = [
    (60, 150),
    (150, 300),
    (2000,3000)
]

# -------------------------------
# Bandpass filter
# -------------------------------
def bandpass_filter(signal, fs, lowcut, highcut, order=4):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return filtfilt(b, a, signal)

# -------------------------------
# Process all CSV files
# -------------------------------
csv_files = glob.glob(os.path.join(FOLDER_PATH, "*.csv"))

all_features = []

for file in csv_files:
    print(f"Processing {file}")
    
    df = pd.read_csv(file)
    if df.shape[1] < 2:
        print(f"Skipping {file}: not enough columns")
        continue

    time = df.iloc[:, 0].values
    accel_g = df.iloc[:, 1].values

    # Convert acceleration to m/s²
    accel = accel_g * G

    dt = np.mean(np.diff(time))
    fs = 1.0 / dt

    file_features = {"file": os.path.basename(file)}

    plt.figure(figsize=(10, 6))
    plt.plot(time, accel, label="Original", alpha=0.5)

    # Apply bands
    for low, high in BANDS:
        band_signal = bandpass_filter(accel, fs, low, high)
        
        label = f"{low}-{high}Hz"
        plt.plot(time, band_signal, label=label)

        # RMS Energy Feature (very useful for ML)
        rms = np.sqrt(np.mean(band_signal**2))
        file_features[f"RMS_{label}"] = rms

    plt.xlabel("Time (s)")
    plt.ylabel("Acceleration (m/s²)")
    plt.title(f"Band Decomposition - {os.path.basename(file)}")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

    plot_path = os.path.join(
        OUTPUT_PLOTS,
        os.path.basename(file).replace(".csv", "_bands.png")
    )
    plt.savefig(plot_path)
    plt.close()

    all_features.append(file_features)

# Save feature summary
features_df = pd.DataFrame(all_features)
features_df.to_csv(OUTPUT_FEATURES, index=False)

print("Processing complete.")
print(f"Feature file saved as: {OUTPUT_FEATURES}")

Processing part_1\11d8b435-ba4a-564f-b0e8-d5cbed8adbb2.csv
Processing part_1\3186c48d-fc24-5300-910a-6d0bafdd87ea.csv
Processing part_1\555cbc73-5a58-53a2-b432-c415f46e8c7c.csv
Processing part_1\6dbf3276-3d5a-5c9f-930e-09da6ec60243.csv
Processing part_1\75a0970d-7c9a-5fd4-9a83-80cddf68ce6c.csv
Processing part_1\771e32b1-39b4-5a58-bb2b-c618ce2701d8.csv
Processing part_1\82e91f2f-4ed5-5591-9617-0ff6f3b0e0c1.csv
Processing part_1\9da3a9bb-65e4-5899-9280-cdd730913e87.csv
Processing part_1\ad57d6b2-f816-5bb2-b4e8-191404207168.csv
Processing part_1\b3f7bc7a-5414-5112-9028-49f4fd4a9072.csv
Processing part_1\b4cbcfe4-09db-5bd3-ae42-c6bf0ab67b91.csv
Processing part_1\ccd17931-56bc-5470-8a47-89356b267edd.csv
Processing part_1\ce31ebce-aa58-5112-9643-89c4559dd5ae.csv
Processing part_1\e45bb50b-c9e8-560b-bb90-1946e68be430.csv
Processing part_1\f26b0d46-fb3e-5a2f-9121-73653390cb09.csv
Processing complete.
Feature file saved as: band_features.csv


In [6]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt


# -------------------------------
# Configuration
# -------------------------------
FOLDER_PATH = "part_1"
OUTPUT_PLOTS = "outputs/band_plots"
OUTPUT_FEATURES = "band_features.csv"

os.makedirs(OUTPUT_PLOTS, exist_ok=True)

G = 9.81  # Convert g to m/s²

# Define frequency bands (customize here)
BANDS = [
    (2000,3000),
    (3000,4000),
    (5000,6000)
]

# -------------------------------
# Bandpass filter
# -------------------------------
def bandpass_filter(signal, fs, lowcut, highcut, order=4):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return filtfilt(b, a, signal)

# -------------------------------
# FFT function
# -------------------------------
def compute_fft(signal, fs):
    n = len(signal)
    fft_vals = np.fft.fft(signal)
    freqs = np.fft.fftfreq(n, 1/fs)

    positive = freqs > 0
    return freqs[positive], np.abs(fft_vals[positive])*2/n

# -------------------------------
# Process all CSV files
# -------------------------------
csv_files = glob.glob(os.path.join(FOLDER_PATH, "*.csv"))

all_features = []

for file in csv_files:
    print(f"Processing {file}")
    
    df = pd.read_csv(file)
    if df.shape[1] < 2:
        print(f"Skipping {file}: not enough columns")
        continue

    time = df.iloc[:, 0].values
    accel_g = df.iloc[:, 1].values
    accel = accel_g * G

    dt = np.mean(np.diff(time))
    fs = 1.0 / dt

    file_features = {"file": os.path.basename(file)}

    # Create figure with subplots
    fig, axes = plt.subplots(len(BANDS), 2, figsize=(12, 3 * len(BANDS)))

    for i, (low, high) in enumerate(BANDS):
        band_signal = bandpass_filter(accel, fs, low, high)
        label = f"{low}-{high}Hz"

        # --- Time domain plot ---
        axes[i, 0].plot(time, band_signal)
        axes[i, 0].set_title(f"{label} - Time Domain")
        axes[i, 0].set_xlabel("Time (s)")
        axes[i, 0].set_ylabel("Acceleration (m/s²)")
        axes[i, 0].grid(True)

        # --- FFT plot ---
        freqs, amplitude = compute_fft(band_signal, fs)
        axes[i, 1].plot(freqs, amplitude)
        axes[i, 1].set_title(f"{label} - FFT")
        axes[i, 1].set_xlabel("Frequency (Hz)")
        axes[i, 1].set_ylabel("Amplitude")
        axes[i, 1].grid(True)

        # --- RMS feature ---
        rms = np.sqrt(np.mean(band_signal**2))
        file_features[f"RMS_{label}"] = rms

    plt.tight_layout()

    plot_path = os.path.join(
        OUTPUT_PLOTS,
        os.path.basename(file).replace(".csv", "_bands_fft.png")
    )
    plt.savefig(plot_path)
    plt.close()

    all_features.append(file_features)

# Save feature summary
features_df = pd.DataFrame(all_features)
features_df.to_csv(OUTPUT_FEATURES, index=False)

print("Processing complete.")
print(f"Feature file saved as: {OUTPUT_FEATURES}")

Processing part_1\11d8b435-ba4a-564f-b0e8-d5cbed8adbb2.csv
Processing part_1\3186c48d-fc24-5300-910a-6d0bafdd87ea.csv
Processing part_1\555cbc73-5a58-53a2-b432-c415f46e8c7c.csv
Processing part_1\6dbf3276-3d5a-5c9f-930e-09da6ec60243.csv
Processing part_1\75a0970d-7c9a-5fd4-9a83-80cddf68ce6c.csv
Processing part_1\771e32b1-39b4-5a58-bb2b-c618ce2701d8.csv
Processing part_1\82e91f2f-4ed5-5591-9617-0ff6f3b0e0c1.csv
Processing part_1\9da3a9bb-65e4-5899-9280-cdd730913e87.csv
Processing part_1\ad57d6b2-f816-5bb2-b4e8-191404207168.csv
Processing part_1\b3f7bc7a-5414-5112-9028-49f4fd4a9072.csv
Processing part_1\b4cbcfe4-09db-5bd3-ae42-c6bf0ab67b91.csv
Processing part_1\ccd17931-56bc-5470-8a47-89356b267edd.csv
Processing part_1\ce31ebce-aa58-5112-9643-89c4559dd5ae.csv
Processing part_1\e45bb50b-c9e8-560b-bb90-1946e68be430.csv
Processing part_1\f26b0d46-fb3e-5a2f-9121-73653390cb09.csv
Processing complete.
Feature file saved as: band_features.csv


In [7]:
features_df

,file,RMS_2000-3000Hz,RMS_3000-4000Hz,RMS_5000-6000Hz
0,11d8b435-ba4a-564f-b0e8-d5cbed8adbb2.csv,0.505110,3.081161,0.014282
1,3186c48d-fc24-5300-910a-6d0bafdd87ea.csv,0.350355,0.082444,0.028676
2,555cbc73-5a58-53a2-b432-c415f46e8c7c.csv,5.679217,10.441604,9.278827
3,6dbf3276-3d5a-5c9f-930e-09da6ec60243.csv,4.874266,15.486310,0.044343
4,75a0970d-7c9a-5fd4-9a83-80cddf68ce6c.csv,1.248741,47.427983,5.551424
5,771e32b1-39b4-5a58-bb2b-c618ce2701d8.csv,7.439847,8.466572,0.015186
6,82e91f2f-4ed5-5591-9617-0ff6f3b0e0c1.csv,11.419981,6.178049,0.818724
7,9da3a9bb-65e4-5899-9280-cdd730913e87.csv,2.979395,8.562491,0.030761
8,ad57d6b2-f816-5bb2-b4e8-191404207168.csv,2.595936,0.998246,15.545378
9,b3f7bc7a-5414-5112-9028-49f4fd4a9072.csv,3.293194,2.861266,0.680410
